In [1]:
import fonction as f
import numpy as np
import os 
from IPython.display import display, Audio
import time as time

In [11]:
from scipy.io.wavfile import write

In [2]:
import matplotlib.pyplot as plt

In [3]:
from pyDOE import lhs

def latin_hypercube_sample(n, dim, bounds):
    """
    Generate a Latin Hypercube sample of size n in dim dimensions, with bounds on each dimension.
    :param n: Number of samples to generate.
    :param dim: Number of dimensions.
    :param bounds: List of tuples specifying the lower and upper bounds for each dimension.
    """
    # Normalize the bounds
    normalized_bounds = [(b[0] - (b[1] - b[0]) / 2, (b[1] - b[0]) / 2) for b in bounds]

    # Generate the Latin Hypercube sample
    samples = lhs(dim, samples=n, criterion='center')

    # Scale and translate the samples to the correct bounds
    for i in range(dim):
        samples[:, i] = samples[:, i] * normalized_bounds[i][1] + normalized_bounds[i][0]

    return samples


In [12]:
N_point = 10

##corde article: 

T_min = 73.9 - 2
T_max = 73.9 + 2

rho_l_min = (3.61 - 0.12) * 10**(-3)
rho_l_max = (3.61 + 0.12) * 10**(-3)

Lc_min = 0.65 - 0.02
Lc_max = 0.65 + 0.02 

r_min = (0.40 - 0.02) * 10e-4
r_max = (0.40 + 0.02) * 10e-4

masseC_min = rho_l_min * Lc_min
masseC_max = rho_l_max * Lc_max

I_min =  masseC_min * r_min **2 / 2
I_max = masseC_max * r_max **2 / 2

B_min =  (4 - 0.3) * 10**(-5) 
B_max =  (4 + 0.3) *10**(-5)

E_corde_min = B_min / I_min
E_corde_max = B_max / I_max

#table composite :

masseT_min = (236e-3 - 0.03)
masseT_max = (236e-3 + 0.03)

L_xmin = 0.40 + 0.02
L_xmax = 0.40 + 0.02

L_ymin = 0.259 - 0.02 #largeur de la table (m)
L_ymax= 0.259 + 0.02

h_min = 2.8e-3 - 0.1e-3
h_max = 2.8e-3 + 0.1e-3

rhoT_min = masseT_min / (L_xmax * L_ymax * h_max)  #masse volumique composite
rhoT_max = masseT_max / (L_ymin * h_min * L_xmin)  #masse volumique composite

nu_min = 0.2 - 0.03
nu_max = 0.2 + 0.03

ET_min = 7e9 - 0.05e-9
ET_max = 7e9 + 0.05e-9


In [13]:
bounds = [(T_min,T_max),(rho_l_min,rho_l_max),(Lc_min,Lc_max), ( E_corde_min,E_corde_max),(I_min,I_max),(h_min,h_max),(nu_min,nu_max),
           (ET_min,ET_max),(rhoT_min,rhoT_max),(L_xmin,L_xmax),(L_ymin,L_ymax)]

XXX = latin_hypercube_sample(N_point,11,bounds)

In [20]:
def creer_son(T,rho_l,L,E_corde,I,h,nu,E_table,rhoT,Lx,Ly,nom_fichier):
    
    Fe = 40000
    MS, CS, KS, phiS_Nx_NmS, NmS, xS = f.corde(T,rho_l,L,E_corde,I) 
   #tic = time.time()
    MB_mod, CB_mod, KB_mod, phiB_NxNy_NmB_mod, NmB_mod, x, y= f.plaque_model(h,nu,E_table,rhoT,Lx,Ly)
    #tic2 = time.time()
    t, FextS_NxS_Nt = f.Simu_config(xS, Fe, T=3)
    #tic3 = time.time()
    M_mod, K_mod, C_mod = f.créer_matrice(MS,MB_mod,KS,KB_mod,CS,CB_mod)
    #tic4 = time.time()
    W_mod_a2, Z_mod_a2 = f.UK_params(M_mod,NmS, NmB_mod, phiS_Nx_NmS, phiB_NxNy_NmB_mod, article = False, model = True, mode = 'A2',x = x, y = y)
    #tic5 = time.time()
    Q, F = f.lounch_simu_article(t,FextS_NxS_Nt,phiS_Nx_NmS,NmS,NmB_mod,M_mod,C_mod,K_mod,Z_mod_a2,W_mod_a2)
    #tic6 = time.time()

    FS = F[:NmS,:]
    #FB = F[NmS:,:]
    FS_NxS_Nt = phiS_Nx_NmS @ FS
    #FB_NxNy_Nt = phiB_NxNy_NmB_mod @ FB
    #tic7 = time.time()
    write("1ier_essai/"+ nom_fichier + '.wav',rate=Fe,data = FS_NxS_Nt[-1,:])
    #display(Audio(FS_NxS_Nt[-1,:], rate=Fe))
    

In [21]:
for i in range(N_point):
    T,rho_l,L,E_corde,I,h,nu,E_table,rhoT,Lx,Ly = XXX[i]
    creer_son(T,rho_l,L,E_corde,I,h,nu,E_table,rhoT,Lx,Ly,str(i))
    

Fréquence du dernier mode de plaque calculé : 1631 Hz
Fréquence du dernier mode de plaque calculé : 1817 Hz
Fréquence du dernier mode de plaque calculé : 1774 Hz
Fréquence du dernier mode de plaque calculé : 1686 Hz
Fréquence du dernier mode de plaque calculé : 1768 Hz
Fréquence du dernier mode de plaque calculé : 1721 Hz
Fréquence du dernier mode de plaque calculé : 1920 Hz
Fréquence du dernier mode de plaque calculé : 1755 Hz
Fréquence du dernier mode de plaque calculé : 1982 Hz
Fréquence du dernier mode de plaque calculé : 1830 Hz


In [ ]:
T,rho_l,L,E_corde,I,h,nu,E_table,rhoT,Lx,Ly = XXX[0]
creer_son(T,rho_l,L,E_corde,I,h,nu,E_table,rhoT,Lx,Ly)

In [ ]:
dico = {}:
for i in range(N_point):
    kle = str(i) + 
    dico{}
    